![banner](https://bccfilkom.net/static/assets/images/BCC-Logo.svg)

# Day 2
### BCC 2023

> Evan \
> Biyu

## Review Day 1

## Intro Machine Learning

### Apa itu ML?
metode untuk menyelesaikan masalah tanpa harus menjabarkan aturan secara eksplisit. Machine learning dapat belajar dan beradaptasi melalui data yang diberikan. 

![wiml](https://i.imgur.com/guZJACw.png)

### Tipe Machine Learning

Berdasarkan Cara Belajar

1. Supervised -> Diberi contoh yang bener kayak gimana
2. Unsupervised -> Belajar sendiri
3. dll.. 

Berdasarkan task-nya
1. Regresi -> saham, curah hujan, harga bawang
2. Klasifikasi -> tipe, kelas

## Data Preprocessing

### Scaling

### Feature Transformation

### Split

## Regresi

### Cara Kerja
![](https://miro.medium.com/v2/resize:fit:800/1*nhGPRU12caIw7NK5Rr3p-w.gif) \
source: https://medium.com/swlh/from-animation-to-intuition-linear-regression-and-logistic-regression-f641a31e1caf

### Training

### Evaluasi

## Klasifikasi

### Cara Kerja
![](https://miro.medium.com/v2/resize:fit:528/0*F1leAlz7TPrVJHY_.gif) \
source: https://saichandra1199.medium.com/classification-vs-regression-622b83ff8e90

### Training

### Evaluasi

## Interpret Model

### SHAP

### Apakah menjawab masalah?

## Tugas 😨